In [1]:
import tracemalloc

from utils_v1 import *


In [2]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates,
    PA, PADeforestationTime, PADistanceMap, PAMultipleDates,
    MTDeforestationTime,
    MTMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential

from src.trainer.evidential import TrainerEvidential

from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 
import src.trainer.evidential as evidential
from src.backend.Logger import Logger



In [3]:
'''
if config["training"] == True:
	dates = [2017, 2018]
else:
	dates = [2018, 2019]
if config["training"] == True:
	dates = [2018, 2019]
else:
	dates = [2019, 2020]
'''

site = 'PA'
if site == 'PA':
	#dates = [2018, 2019]
	dates = [2018, 2019]

	dataset = PAMultipleDates(dates = dates, # dates = [2016,2017,2018,2019]
		addPastDeforestationInput = True,
		borderBuffer = 2)			

	#exp_id_base = 148
	# exp_id_base = 155
	exp_id_base = 194
	exp_id_base = 198
	exp_id_base = 214

	# earlier date
	exp_id_base = 224
	exp_id_base = 194


else:
	
	dates = [2019, 2020]

	dataset = MTMultipleDates(dates = dates, # dates = [2016,2017,2018,2019]
		addPastDeforestationInput = True,
		borderBuffer = 2)	
	
	# MT
	# exp_id_base = 196

ic(dates)

if issubclass(type(dataset), MultipleDates):
	patchesHandler = PatchesHandlerEvidential(
		dataset)
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
ic| dates: [2018, 2019]
ic| self.input_image_shape: 21
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]


In [4]:
repetition_n = 10
experiment_values = np.ones(repetition_n).astype(np.uint8)


# experiment_values = [int(x) for x in experiment_values]
results = []



exp_ids = range(exp_id_base, exp_id_base+len(experiment_values))


In [5]:
exp_ids

range(194, 204)

In [6]:
from enum import Enum
UncertaintyMethod = Enum('Predictor', 'pred_var MI pred_entropy KL pred_entropy_single evidential')
uncertaintyMethod = "evidential"


In [7]:
idx = 0
while idx < len(exp_ids):
# for experiment_value, exp_id in zip(experiment_values, exp_ids):
    try:
        config = {
        "training": False,
        "learning_rate": 1e-4,
        "weights": [0.1, 0.9, 0],
        "save_probabilities": False,
        "loadInference": False,
        "addPastDeforestationInput": True,
        "inference_times": 1,
        "Uncertainty": {
            "annealing_step": 10*375 # 1875
        },
        "uncertainty_method": uncertaintyMethod,
        "evidence_function": evidential.relu_evidence
        }
        print("Beginning run number {}".format(idx))

        logger = Logger()
        trainer = TrainerEvidential(config, dataset, patchesHandler, logger)

        trainer.defineExperiment(exp_ids[idx])

        trainer.setExperimentPath()
        trainer.createLogFolders()

        trainer.loadDataset()

        # %%
        if config["training"] == True:
            trainer.run()

        # %%
        result = trainer.run_predictor()
        print("Result idx {}: {}".format(idx, result))
        results.append(result)


        with open('log.pkl', 'wb') as f:
            pickle.dump(results, f)
        idx += 1
        print("Grid execution idx: {}".format(idx))
    except Exception as e: 
        print(e)
        print("Execution error in idx {}. Retrying...".format(idx))
        continue

Beginning run number 0
D:/Jorge/datasets/deforestation/Para/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Label where deforestation past years is actual date (2019) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

Past deforestation before 2008 is 2
D:/Jorge/datasets/deforestation/deforestation_before_2008/deforestation_before_2008_para.tif


ic| np.unique(label_past_deforestation_before_2008, return_counts=True): (array([  0, 215], dtype=uint8), array([124211220,  38957970], dtype=int64))
ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))
ic| label_per_date.shape: (17730, 9203, 1)


(array([0., 1.], dtype=float32), array([162296140,    819860], dtype=int64))
[0. 1.]


ic| self.label_mask.shape: (17730, 9200)
ic| self.mask_amazon_ts.shape: (17730, 9200)
ic| self.label_mask_test.shape: (81558000,)
ic| self.error_mask_test.shape: (53942169,)
ic| trainer.dataset.previewLims1: array([2200, 3200, 6900, 7900])
    trainer.dataset.previewLims2: array([ 500, 1500, 3500, 4500])
ic| np.unique(trainer.mask_amazon_ts[lims[0]:lims[1], lims[2]:lims[3]]): array([1], dtype=uint8)
ic| np.unique(trainer.mask_amazon_ts[lims[0]:lims[1], lims[2]:lims[3]], return_counts=True): (array([1], dtype=uint8), array([1000000], dtype=int64))
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(53942169,) (53942169,)
0.2857458654145891


In [ ]:
print(results)

[{'other_uncertainty_metrics': {'sUEO': 0.3670126015642983, 'ece_score': 0.04906518334555479}, 'uncertainty_result': {'metrics': {'precision_L': array([0.8460478]), 'recall_L': array([0.92707223]), 'recall_Ltotal': array([0.55478324]), 'AA': array([0.03000072]), 'precision_H': array([nan]), 'recall_H': array([0.]), 'UEO': array([0.23447353]), 'f1_L': array([0.88470877]), 'f1_H': array([nan]), 'f1': 67.01}, 'metrics_audited': {'precision': array([0.90452009]), 'recall': array([0.9563582]), 'f1': array([0.92971712])}, 'exp': 196}}, {'other_uncertainty_metrics': {'sUEO': 0.0, 'ece_score': 0.01947295218764906}, 'uncertainty_result': {'metrics': {'precision_L': array([nan]), 'recall_L': array([0.]), 'recall_Ltotal': array([0.]), 'AA': array([0.03435077]), 'precision_H': array([nan]), 'recall_H': array([0.]), 'UEO': array([0.54008159]), 'f1_L': array([nan]), 'f1_H': array([nan]), 'f1': 0.0}, 'metrics_audited': {'precision': array([1.]), 'recall': array([0.96602314]), 'f1': array([0.98271797]

In [ ]:
print(len(results))

10


In [ ]:
results[9]

{'other_uncertainty_metrics': {'sUEO': 0.5404739587669327,
  'ece_score': 0.03641546979657238},
 'uncertainty_result': {'metrics': {'precision_L': array([0.92989761]),
   'recall_L': array([0.93764161]),
   'recall_Ltotal': array([0.503057]),
   'AA': array([0.02999948]),
   'precision_H': array([0.58242773]),
   'recall_H': array([0.18413217]),
   'UEO': array([0.26910476]),
   'f1_L': array([0.93375355]),
   'f1_H': array([0.27980509]),
   'f1': 69.74},
  'metrics_audited': {'precision': array([0.96224463]),
   'recall': array([0.9665439]),
   'f1': array([0.96438947])},
  'exp': 205}}

In [ ]:
trainer.f1

69.74

In [ ]:
print(trainer.f1)

69.74
